# Notes

Task: Please review the dataset and start outlining the way you would go about the model-building process with the goal of predicting the probability that a player will win a foul on his possession after he receives or recovers the ball.

Notes:
- In the case of a received pass, the previous event (related event) is to be inspected for the pass attributes (location, length, highness...)
- Aggregated player data could be useful (ratio of passes under pressure signaling if he waits for pressure, dribbles per match)
- Referees are really important
- Home or away
- (Game state)


Cooridnates
- I suspect the origo is different for the two teams competing

# Dependencies

In [ ]:
import pandas as pd
import numpy as np

from typing import Tuple
from mplsoccer import VerticalPitch, add_image, FontManager, Sbopen, Pitch
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.utils import resample
from hdbscan import HDBSCAN
from hyperopt import atpe, fmin, hp
from umap.umap_ import UMAP
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

## Custom functions

In [ ]:
def extract_location(location_col: pd.Series) -> Tuple[pd.Series, pd.Series]:
    """
    The main purpose of this function is to bring the location columns to usable form
        Inputs:
            location_col (pd.Series): Pandas series containing location data.
        
        Returns:
            Tuple[pd.Series, pd.Series]: Extracted X, Y coordinates as two series.
    """
    splitted_cols = location_col.str.split(',', expand=True)
    loc_x, loc_y = splitted_cols.iloc[:, 0], splitted_cols.iloc[:, 1]
    loc_x = pd.to_numeric(loc_x, errors='coerce')
    loc_y = pd.to_numeric(loc_y, errors='coerce')
    
    return loc_x, loc_y

def add_in_M3(input_df: pd.DataFrame, x_name: str='x') -> pd.Series:
    return ((input_df[x_name]<=80) & (input_df[x_name]>=40)).astype(int)

def add_in_DBox(input_df: pd.DataFrame, x_name: str='x', y_name: str='x') -> pd.Series:
    return (((input_df[x_name]<=15) & (input_df[y_name]>=33) & (input_df[y_name]<=47) & (input_df[x_name]>=2))).astype(int)

def add_forward_pass(input_df: pd.DataFrame, pass_angle_name: str='pass.angle') -> pd.Series:
    return (input_df[pass_angle_name].between(-np.pi/4, np.pi/4)).astype(int)

def plot_histograms_grid(df, category_col, bins=None):
    # Number of columns (excluding category column)
    num_cols = len(df.columns) - 1  # Exclude the category column
    num_rows = int(np.ceil(num_cols / 4))  # Calculate number of rows needed for 4 columns per row

    fig, axes = plt.subplots(num_rows, 4, figsize=(15, 3 * num_rows))  # Create grid
    
    # Flatten axes for easy indexing
    axes = axes.flatten()
    
    for idx, col in enumerate(df.drop(columns=[category_col]).columns):
        ax = axes[idx]
        
        # Drop NaN values from the current column
        df_col = df.dropna(subset=[col])
        df_col = df.loc[df[col]!='NONE']

        # Plot histograms for each category in the column
        ax.hist(df_col[df_col[category_col] == 0][col],
                bins=bins,
                alpha=0.7, label='No foul', color='grey', density=True)
        ax.hist(df_col[df_col[category_col] == 1][col],
                bins=bins,
                alpha=0.5, label='Foul won', color='red', density=True)
        
        ax.set_title(col)  # Set title as column name
        ax.legend(prop={'size': 8})  # Adjust legend size
        ax.set_yticks([])
        ax.set_ylabel("")
    
    # Hide empty subplots if any
    for i in range(idx + 1, len(axes)):
        fig.delaxes(axes[i])
    

    plt.tight_layout()
    plt.show()

def plot_histogram_one(df_visu, category_col, content_col=None, density=False, bins=None):
    content_col = category_col if content_col is None else content_col
    df_visu = df_visu.dropna(subset=[category_col])  # Drop NaN values from the column
    
    plt.figure(figsize=(6, 4))
    
    # Plot histograms for each category
    plt.hist(df_visu[df_visu[category_col] == 0][content_col], alpha=0.7, label='No foul', color='grey',
             density=density, bins=bins)
    plt.hist(df_visu[df_visu[category_col] == 1][content_col], alpha=0.5, label='Foul won', color='red',
             density=density, bins=bins)

    plt.title(content_col)
    plt.legend(prop={'size': 8})
    plt.yticks([])
    plt.ylabel("")

    plt.show()


## Configs

In [ ]:
to_boolean_cols = [
    'under_pressure',
    'under_pressure_related',
    'counterpress',
    'contains_foulwon'
]

to_string_cols = [
    'type.id',
    'play_pattern.id',
    'play_pattern.id_related',
    'team.id',
    'opponent_team.id',
    # 'team.id_related',  # useful with having opponent and own team id?
    'player.id',
    'position.id',
    'position.id_related',
    'pass.body_part.id',
    'pass.body_part.id_related',
    'pass.type.id',
    'pass.type.id_related',
    'duel.outcome.id',
    'stadium.id',
    'referee.id',
    'manager_id_opponent',
    'manager_id_own',
    'manager_countryid_opponent',
    'manager_countryid_own',
    'location_x_bin',
    'location_y_bin',
    'location_y_bin_width',
    'pass.height.id',
    'pass.height.id_related',
]

to_roundup_cols = [
    'pass.length',
    'pass.length_related',
    # 'location_x',
    # 'location_y'
]

# to_roundup2_cols = [
#     'pass.angle',
#     'pass.angle_related',
# ]

# Data

## Data import

In [ ]:
df_event = pd.read_csv('/Users/thomasregos/Owny/Assessment/Swish_Take_Home_Assessment_Data_-_EPL/epl_event_data_15.csv',
                    #    nrows=10000
                       )

df_matches = pd.read_csv('/Users/thomasregos/Owny/Assessment/Swish_Take_Home_Assessment_Data_-_EPL/epl_matches_15.csv')

## Data exploration

In [ ]:
df_matches.columns

In [ ]:
df_event['type.name'].unique()

In [ ]:

# n = 1528

# bbb = df_event.iloc[max(n - 10, 0): min(n + 11, len(df_event))]

# Training dataset
The main goal is to create a table where all the starting actions of a mini possession is listed. Mini possession is part of a possession where all the actions are taken by the same player. These events then should be ammended with a target variable which signals that a mini possession contained a foul won or not. Additional variables are later added for more info about the event.

## Skeleton of the training dataset



Each row should be a mini possession of the player, which starts by receiving or recovering the ball. In order to identify the mini possessions a number is assigned to a player's events inside a possession. This number is different for all of the player's mini possessions. So if he received a pass then carried the ball and passed it, it will receive a number and then if inside the possession he receives the ball again these events will have a different number. Combining these mini possession IDs with the possession ID, match ID and the player ID all the mini possession can be identified.



Notes:

- Mini possessions have to be detected at player level
    - First the sub possession should be calculated and then create a matchid_possession_subpossession 
- For our purpose those mini possessions are relevant that start with ball recovery or received pass 
    - Valid mini possessions are identified by type.name as we have a better control this way. For detecting ball recovery it would be an option to check if the possession_team.name has changed but there are examples of making a recovery winning a foul and the data still says that the possession is on the other team (id: 7317). As the possession mechanics are not clear I will use type.name.
    - Sometimes a possession starts with a 'Foul Won'. I don't think we should include those as the probability of these possessions to contain a 'Foul Won' is certain.
    - type.name == 'Ball Recovery'
    - pass.type.name = 'Recovery'
        - If a recovery is also a pass. These could be also relevant if the pass provokes a hand ball.
    - type.name == 'Ball Receipt*'
        - Beware, there is an event for receiving an incomplete pass which was already intercepted, with type.name = 'Ball Receipt*'
        - ball_receipt.outcome.id = 9 (incomplete pass that was actually not received)
        - These should be None as they are not real mini possessions
    - type.name == 'Interception'
        - Although it is not a ball recovery by name and no instruction is given to include these, but I think this action recovers the ball in open play so it satisfies our criteria
    - type.name == 'Duel'
        - Although it is not a ball recovery by name and no instruction is given to include these, but I think this action recovers the ball in open play so it satisfies our criteria
    - type.name == 'Goal Keeper'
        - Although it is not a ball recovery by name and no instruction is given to include these, but I think this action recovers the ball in open play so it satisfies our criteria
    - type.name == 'Clearance'
        - Although it is not a ball recovery by name and no instruction is given to include these, but I think this action recovers the ball in open play so it satisfies our criteria
    - type.name == 'Block'
        - Although it is not a ball recovery by name and no instruction is given to include these, but I think this action recovers the ball in open play so it satisfies our criteria
- Getting rid of Pressure events
    - From our perspective it is enough to know if certain events were under pressure or not and it is reflected in the 'under_pressure' column
    - Getting rid of the individual 'Pressure' events enables us to have a chronological event chain for player possessions
- Getting rid of 'Foul Committed' events
    - type.name == 'Foul Committed'
    - Foul won is enough for our purpose
- Foul won
    - type.name == 'Foul Won'
    - It can happen that a player not having the ball draws the foul. (index: 646)
    - It can happen that a player having the ball commits a foul. (index: 898)
    - It might happen that a foul is won but the play goes on as I saw several examples of winning a foul and then having a carry event that starts at the same location. It is supported by the fact that this carry and the subsequent events are not labeled as 'play_pattern.name' = 'From Free Kick'. These could be when the referee plays advantage. As there is no info about how to consider these fouls that were won but ultimately not resulted in a freekick I will count them as a foul won. (index: 1012)


In [ ]:
# Mini possessions
df_filtered = df_event.copy()

# - Getting rid of unneccessary rows
notvalid_start_events = ['Foul Committed', 'Pressure', 'Dribbled Past']
df_filtered = df_filtered[~df_filtered['type.name'].isin(notvalid_start_events)]

# - calculating mini_possessions inside possessions
# not perfect but we will be able to get a unique identifier for each mini possession
df_filtered['poss_mini'] = (
    df_filtered.groupby('possession')['player.id']
    .transform(lambda x: x.ne(x.shift()).cumsum())
)

# - adding unique identifier for mini possessions
df_filtered['poss_mini_id'] = df_filtered[['match_id', 'player.id', 'possession', 'poss_mini']].fillna('_').apply(lambda row: '_'.join(row.apply(lambda x: str(int(x)) if isinstance(x, float) and x.is_integer() else str(x))), axis=1)


# - identifying mini possessions that started with a ball recovery or a ball receipt
valid_start_events = ['Ball Recovery', 'Ball Receipt*', 'Duel', 'Interception', 'Goal Keeper', 'Clearance', 'Block']

df_filtered['valid_start'] = df_filtered.groupby('poss_mini_id')['type.name'].transform(
    lambda x: 1 if x.isin(valid_start_events).any() or (df_filtered.loc[x.index, 'pass.type.name'] == 'Recovery').any() else 0
)

# - identifying mini possession that contains Foul Won
df_filtered['contains_foulwon'] = df_filtered.groupby('poss_mini_id')['type.name'].transform(lambda x: 1 if 'Foul Won' in x.values else 0)


# - preserving only starting events of mini possessions
mask = ((df_filtered['valid_start']==1))
df_filtered = df_filtered[mask]

df_training_full = df_filtered.drop_duplicates(subset=['poss_mini_id'], keep='first')


## Additional info about the events



I only use data that is available at the time of the ball recovery or pass reception.

### Info about prior events

#### Checking the type of related events

Info about opponent or teammate passes might be useful.

Notes:
- Most of the related events are passes from a teammate or an opponent pass.
- Pressures are also above 1% but this info is reflected on the starting event's record itself.
- Opponent shots are around 2% of the whole related events, and they might be interesting for niche events, but not used at this time.


In [ ]:
df_filtered_event = df_event.copy()
df_filtered = df_training_full.copy()

df_filtered_event['seconds_elapsed'] = df_filtered_event['minute'] * 60 + df_filtered_event['second']
df_filtered['seconds_elapsed'] = df_filtered['minute'] * 60 + df_filtered['second']

# Expanding related events into separate rows
df_exploded = df_filtered.assign(related_events=df_filtered['related_events'].str.split(', ')).explode('related_events')

# Merging with original data to get the related event details
df_merged = df_exploded.merge(df_filtered_event, left_on='related_events', right_on='id', how='left', suffixes=['', '_related'])

# Keeping only related events that happened before
df_valid = df_merged[df_merged['seconds_elapsed_related'] < df_merged['seconds_elapsed']]

# Differentiating between opponent and own related events
df_valid.loc[df_valid['team.id'] != df_valid['team.id_related'], 'type.name_related'] = (df_valid['type.name_related'].astype(str) + "_opp")

# Ratio of certain event types in the valid related events
rel_counts = df_valid['type.name_related'].value_counts(normalize=True)
print(rel_counts)


#### Including data about related events

In [ ]:
# preserving only the desired type of related events
df_valid = df_valid[df_valid['type.name_related'].isin(['Pass', 'Pass_opp'])]

# renaming important columns
df_valid = df_valid.rename(columns={'related_events': 'relevant_related_id'})

In [ ]:

df_filtered = df_training_full.copy()

df_filtered['seconds_elapsed'] = df_filtered['minute'] * 60 + df_filtered['second']

# checking if there are no two relevant related events for an event
print(df_valid['id'].nunique() == df_valid.shape[0])

# merging the relevant related event info to the training dataset
# df_filtered = df_filtered.merge(df_valid[['seconds_elapsed', 'type.name_related', 'relevant_related_id', 'id']], how='left', on='id')
df_filtered = df_filtered.merge(df_valid[['type.name_related', 'relevant_related_id', 'id']], how='left', on='id')

# merging on additional info of the relevant event from the main data
relevant_event_cols = [
    'location',
    'under_pressure',
    'play_pattern.id',
    'team.id',
    'position.id',
    'pass.length',
    'pass.angle',
    'pass.height.id',
    'pass.body_part.id',
    'pass.type.id',
    'pass.end_location',
    ]

df_filtered = df_filtered.merge(df_event[relevant_event_cols + ['id']],
                                how='left',
                                left_on='relevant_related_id', right_on='id',
                                suffixes=['', '_related'])

# getting dummies for 'type.name_related'
df_filtered = pd.concat([df_filtered, pd.get_dummies(df_filtered['type.name_related'], prefix='type.name_related', dtype=int)], axis=1)  # number of categories are fixed by above filtering
df_filtered = df_filtered.drop(columns=['type.name_related'])


df_training_full = df_filtered.copy()


### Info about the teams, the match and the referee

I include info about the team's manager, the opponent team's manager, the referee and the venue.

Notes:
 - Many columns could be useful but contain the same value like 'competition.country_name' or 'referee.country.id'.

In [ ]:
df_filtered = df_training_full.copy()
df_filtered_matches = df_matches.copy()


# - creating manager sub table
# -- there is an error which is manually checked and corrected
# source: https://www.soccerbase.com/teams/team.sd?team_id=2513&teamTabs=managers
# on matchid: 3754151 the correct manager for home_team.managers: 3626, Francesco Guidolin, NA, 1955-10-03, 112, Italy
df_filtered_matches.loc[df_filtered_matches['match_id'] == 3754151, 'home_team.managers'] = '3626, Francesco Guidolin, NA, 1955-10-03, 112, Italy'
# on matchid: 3753988 the correct manager for away_team.managers: 3626, Francesco Guidolin, NA, 1955-10-03, 112, Italy
df_filtered_matches.loc[df_filtered_matches['match_id'] == 3753988, 'away_team.managers'] = '3626, Francesco Guidolin, NA, 1955-10-03, 112, Italy'

# -- Splitting the column into multiple parts
exploded_managers = ['manager_id', 'manager_name', 'manager_nickname', 'manager_birthdate', 'manager_countryid', 'manager_countryname']
df_filtered_matches[[f"{i}_home" for i in exploded_managers]] = df_filtered_matches['home_team.managers'].str.split(', ', expand=True)
df_filtered_matches[[f"{i}_away" for i in exploded_managers]] = df_filtered_matches['away_team.managers'].str.split(', ', expand=True)


# - aggregating 

exploded_managers_mini = ['manager_id', 'manager_birthdate', 'manager_countryid']
relevant_matchinfo_cols = [
    'home_team.home_team_id',
    'away_team.away_team_id',
    'stadium.id',
    'referee.id',
    # 'match_date'
] + [f"{i}_home" for i in exploded_managers_mini] + [f"{i}_away" for i in exploded_managers_mini]

df_filtered = df_filtered.merge(df_filtered_matches[relevant_matchinfo_cols + ['match_id']],
                                how='left',
                                on='match_id')

# Home or away team
df_filtered['played_at_home'] = np.where(df_filtered['team.id'] == df_filtered['home_team.home_team_id'], 1, 0)

# Opponent
df_filtered['opponent_team.id'] = np.where(df_filtered['team.id'] == df_filtered['home_team.home_team_id'], df_filtered['away_team.away_team_id'], df_filtered['home_team.home_team_id'])

# Opponent Manager
opp_manager_exploded = [f"{i}_opponent" for i in exploded_managers_mini]
df_filtered[opp_manager_exploded] = np.where(df_filtered['played_at_home'].values[:, None] == 0,
                                             df_filtered[[f"{i}_home" for i in exploded_managers_mini]].to_numpy(),
                                             df_filtered[[f"{i}_away" for i in exploded_managers_mini]].to_numpy())

# Manager
own_manager_exploded = [f"{i}_own" for i in exploded_managers_mini]
df_filtered[own_manager_exploded] = np.where(df_filtered['played_at_home'].values[:, None] == 1,
                                             df_filtered[[f"{i}_home" for i in exploded_managers_mini]].to_numpy(),
                                             df_filtered[[f"{i}_away" for i in exploded_managers_mini]].to_numpy())

df_training_full = df_filtered.copy()

### Additional feature engineering

Steps
- Location is brought to usable format, and then binned to make it less noisy.
- Widthness is calculated from the location bins.
- Opponent event location switched to same direction as normal events, while preserving normal direction for own team related events.
- Identifying forward passes from angles.
- Calculate the manager's age in days to the first day of the season.
- Checking where the fouls happen and create zone labels for the high probability areas.
- Short passes have a higher probability to result in a foul won and also it is extremely skewed. The median for the passes that resulted in a foul is just around 6.5 and the max is around 26 which are really low numbers considering the max possibble is 120. So it makes sense to label it, but instead I will take the log of it. For related passes the distribution much more resembles the non foul winning pass length distribution, but we will see that after taking the log a difference can be seen in the distributions.

Notes
- The pitch size looks to be 120x80
- Previous pass end location is the same as the event start location, so there is no use for 'pass.end_location_related_y', 'pass.end_location_related_x'

In [ ]:
df_filtered = df_training_full.copy()


# - location

# -- Bringing to usable format
df_filtered['location_x'], df_filtered['location_y'] = extract_location(df_filtered['location'])  # location

# Not included to avoid None values in numerical columns
# df_filtered['pass.end_location_x'], df_filtered['pass.end_location_y'] = extract_location(df_filtered['pass.end_location'])  # pass end
# df_filtered['location_related_x'], df_filtered['location_related_y'] = extract_location(df_filtered['location_related'])  # related event's location
# df_filtered['pass.end_location_related_x'], df_filtered['pass.end_location_related_y'] = extract_location(df_filtered['pass.end_location_related'])  # related event's pass end location
# df_filtered['carry.end_location_x'], df_filtered['carry.end_location_y'] = extract_location(df_filtered['carry.end_location'])  # carry end

# related event location switch but only for opponent actions
# Not included to avoid None values in numerical columns
# df_filtered['location_related_y'] = np.where(df_filtered['type.name_related_Pass_opp'] == 1,
#                                              80 - df_filtered['location_related_y'],
#                                              df_filtered['location_related_y'])

# df_filtered['location_related_x'] = np.where(df_filtered['type.name_related_Pass_opp'] == 1,
#                                              120 - df_filtered['location_related_x'],
#                                              df_filtered['location_related_x'])

# -- Binning location
# bin edges
x_bins = np.linspace(0, 120, 25)
y_bins = np.linspace(0, 80, 17)

# assigning bins
location_cols = ['location',
                #  'pass.end_location', 'location_related'  # Not included to avoid None values in numerical columns
                 ]
for loc_col in location_cols:
    # bins
    df_filtered[f'{loc_col}_x_bin'] = np.where(
        df_filtered[f'{loc_col}_x'].isna(),
        np.nan,
        np.digitize(df_filtered[f'{loc_col}_x'], x_bins, right=False) - 1)
    df_filtered[f'{loc_col}_y_bin'] = np.where(
        df_filtered[f'{loc_col}_y'].isna(),
        np.nan,
        np.digitize(df_filtered[f'{loc_col}_y'], x_bins, right=False) - 1)

    # width
    df_filtered[f'{loc_col}_y_bin_width'] = np.where(
        df_filtered[f'{loc_col}_y_bin'].isna(),
        np.nan, 
        df_filtered[f'{loc_col}_y_bin'].apply(lambda x: max(x, 16 - x)) - 8)

# -- Angle
df_filtered['forward_pass'] = add_forward_pass(df_filtered)

# flipping pass angle for opponent related passes
df_filtered['pass.angle_related'] = np.where(
    df_filtered['type.name_related_Pass_opp']==1,
    (df_filtered['pass.angle_related'] + np.pi) % (2 * np.pi),
    df_filtered['pass.angle_related'])

# calculating if an opponent related pass was a backward pass from the opponent's view
# too few, not used
# df_filtered.loc[df_filtered['type.name_related_Pass_opp'] == 1, 'type.name_related_Pass_opp_backward'] = (
#     df_filtered['pass.angle_related'].between(-np.pi/4, np.pi/4)
# ).astype(int)
# df_filtered['type.name_related_Pass_opp_backward'] = df_filtered['type.name_related_Pass_opp_backward'].fillna(0).astype(int)

# - Manager age in days at time of season start
# Formatting
df_filtered['manager_birthdate_opponent'] = pd.to_datetime(df_filtered['manager_birthdate_opponent'])
df_filtered['manager_birthdate_own'] = pd.to_datetime(df_filtered['manager_birthdate_own'])

# Calculating the difference in days to the first day of the season
target_date = pd.to_datetime(df_matches['match_date'].min())
df_filtered['manager_age_opponent'] = (target_date - df_filtered['manager_birthdate_opponent']).dt.days
df_filtered['manager_age_own'] = (target_date - df_filtered['manager_birthdate_own']).dt.days

# -- Checking if it is close to high prob foul won area (Explanation in next block)
df_filtered['loc_in_M3'] = add_in_M3(df_filtered, 'location_x')
df_filtered['loc_in_DBox'] = add_in_DBox(df_filtered, 'location_x', 'location_y')

# -- Ball reception failure
df_filtered['ball_receiption_failed'] = np.where(df_training_full['ball_receipt.outcome.id'] == 9, 1, 0)

# -- pass type will contain only 1 other than '66'
df_filtered['pass.type.id'] = np.where(df_filtered['pass.type.id']==66, 1, 0)


df_training_full = df_filtered.copy()

#### Where did players win the most foul?
I think it might be useful to check where do players won the most fouls. Based on this, valid starting events that are close to these areas, might have a higher probability of winning a foul on the possession. This is something that models could learn from the starting locations but this could speed up the learning process.

Findings:
- Most fouls were won in the middle third.
- There is a jump in density in front of the own goal. Probably after corners and free kicks there are a lot of fouls won by defenders and especially the goalkeeper.
- These events are fairly symmetrical along the X axis.

In [ ]:
df_filtered = df_event.copy()

# filtering and aggregating data for Foul Won events
df_filtered = df_filtered.loc[df_filtered['type.name'] == 'Foul Won']
df_filtered['location_x'], df_filtered['location_y'] = extract_location(df_filtered['location'])  # location

# visualising
pitch = Pitch(line_color='#000009', line_zorder=2)
fig, ax = pitch.draw(figsize=(6, 8))
kde = pitch.kdeplot(df_filtered['location_x'], df_filtered['location_y'], ax=ax,
                    fill=True,
                    levels=100,
                    thresh=0,
                    cut=4,
                    cmap='inferno')


# highlighting areas
shape1 = np.array([[40, 0], [40, 80], [80, 80], [80, 0]])
shape2 = np.array([[2, 33], [15, 33], [15, 47], [2, 47]])
verts = [shape1, shape2]
pitch.polygon(verts, color='white', alpha=0.4, ax=ax, zorder=4)

txt = ax.text(x=92, y=75, s='Middle third', size=12, color='white', alpha=0.6,
              va='center', ha='center')

txt = ax.text(x=15, y=53, s='Defensive box', size=12, color='white', alpha=0.6,
              va='center', ha='center')

txt = ax.text(x=60, y=-5, s='Fouls Won', size=15, color='black', va='center', ha='center')


### Dropping unnecessary columns

In [ ]:
cols_to_drop = [
    'Unnamed: 0',
    'index',
    'id',
    # 'player.id',  # maybe it would be over reliant?
    'timestamp',
    'second',
    'minute',
    'duration',  # extremely skewed
    'possession',
    'related_events',
    'location',
    'type.name',
    'possession_team.id',
    'possession_team.name',
    'play_pattern.name',
    'team.name',
    'tactics.formation',
    'tactics.lineup',
    'player.name',
    'position.name',
    'pass.height.name',
    'pass.body_part.name',
    'pass.type.name',
    'pass.outcome.name',
    'pass.outcome.id',  # Not known info at the time of ball reception or recovery
    'pass.receipient.id',
    'pass.recipient.name',
    'pass.end_location',
    'dribble.outcome.name',
    'ball_receipt.outcome.id',
    'ball_receipt.outcome.name',
    'carry.end_location',  # only nan
    'duel.outcome.name',
    'competition_id',
    'match_id',
    'poss_mini',
    'poss_mini_id',
    'valid_start',
    'relevant_related_id',
    'location_related',
    'pass.end_location_related',  # same as location for the main event
    'id_related',
    'manager_id_home',
    'manager_id_away',
    'manager_countryid_home',
    'manager_countryid_away',
    'manager_birthdate_home',
    'manager_birthdate_away',
    'manager_birthdate_opponent',
    'manager_birthdate_own',
    'pass.end_location_y',
    'pass.end_location_x',
    'location_related_y',
    'location_related_x',
    # 'pass.end_location_y_bin',  # the event might come in with this
    # 'pass.end_location_x_bin',
    'dribble.outcome.id',  # too few (2)
    'home_team.home_team_id',
    'away_team.home_team_id',
    'team.id_related',  # useful with having opponent and own team id?
    'location_x',  # with bins in it could be not relevant
    'location_y',  # with bins in it could be not relevant
    'pass.angle',  # not used to avoid None inputing
    'pass.angle_related',  # not used to avoid None inputing
]

df_training_full = df_training_full.drop(columns=cols_to_drop, errors='ignore')


### Using proper format for each columns



- Boolean columns
    - Original format is not 0, 1
- ID columns
    - These contain numbers but the models should interpret them as categories
- Rounding up
    - Some values are too detailed hence they are round up to 0 or 2 decimals

In [ ]:
df_filtered = df_training_full.copy()


for col in to_boolean_cols:
    df_filtered[col] = df_filtered[col].fillna(False).astype(int)

for col in to_string_cols:
    df_filtered[col] = df_filtered[col].apply(lambda x: str(int(x)) if pd.notna(x) else np.nan)
    # df_filtered[col] = df_filtered[col].apply(lambda x: str(int(x)) if pd.notna(x) else 'NONE')

for col in to_roundup_cols:
    df_filtered[col] = df_filtered[col].round()

# for col in to_roundup2_cols:
#     df_filtered[col] = df_filtered[col].round(2)

df_training_full = df_filtered.copy()

In [ ]:
# reordering columns to have contains_foulwon at the front
columns = ['contains_foulwon'] + [col for col in df_training_full.columns if col != 'contains_foulwon']
df_training_full = df_training_full[columns]

In [ ]:
df_training_full.shape


In [ ]:
df_training_full.to_csv('/Users/thomasregos/Owny/Assessment/Swish_Take_Home_Assessment_Data_-_EPL/Output/training_data.csv', index=False)

# Modeling

In [ ]:
df_input = pd.read_csv('/Users/thomasregos/Owny/Assessment/Swish_Take_Home_Assessment_Data_-_EPL/Output/training_data.csv')

df_filtered = df_input.copy()

for col in to_boolean_cols:
    df_filtered[col] = df_filtered[col].fillna(False).astype(int)

for col in to_string_cols:
    df_filtered[col] = df_filtered[col].apply(lambda x: str(int(x)) if pd.notna(x) else np.nan)
    # df_filtered[col] = df_filtered[col].apply(lambda x: str(int(x)) if pd.notna(x) else 'NONE')

for col in to_roundup_cols:
    df_filtered[col] = df_filtered[col].apply(lambda x: round(x) if pd.notna(x) else 0)

# for col in to_roundup2_cols:
#     df_filtered[col] = df_filtered[col].round(2)

df_input = df_filtered.copy()

## Independent variables

### Distributions

Let's assess the distribution of each variable accounting for the fact that the event was followed by a won foul or not.

Most of the variables that we eneded up with have a difference visually. However, there are some, where the difference seems to be negligible. Later I will pay extra attention to some of these when assessing their importance in the models. 'pass.type.id' contained '66' as valid value and only one time a different one so I converted is to be a bool column.

There are several variables that back intuition:
- If the event is under pressure than the probability is higher to win a foul
- There are less fouls won in the opening minutes of the games compared to no foul mini possessions
- There is a higher probability of winning a foul in the middle third
- If the event is a pass and it is with the right foot there is a higher probability of resulting in a foul won than for example with a pass that was a header. The intuition behind this is that a pass with the (usually) stronger foot is more precise, more dangerous hence has a higher chance to provoke a foul than a header.
- There are teams that win less fouls than expected based on the number of their mini possessions. The two teams that dive the less or the referess dislike the most is Norwich and Liverpool.
- If a player won a duel it is highly likely that he will be fouled (duel.outcome.id == 4)

Interestingly there is more possibility to win a foul outside the defensive box than inside. I suspect that the local maximum can still be a good indicator even tough globally that probability is not higher than for example in the middle third.

In [ ]:
df_visu = df_input.copy()

for col in to_string_cols:
    df_visu[col] = df_visu[col].apply(lambda x: str(int(x)) if pd.notna(x) else 'NONE')

plot_histograms_grid(df_visu, 'contains_foulwon')

In [ ]:
# pass.type.id

df_filtered = df_input.copy()

pass_typids = df_filtered['pass.type.id'].value_counts()
pass_typeid_66 = df_filtered[df_filtered['pass.type.id'] == '66']['contains_foulwon'].value_counts()

print(f"Only one event contains pass.type.id == 61: {pass_typids}")

In [ ]:
# Teams winning less fouls then expected

df_filtered = df_event.copy()
df_teams = df_filtered.groupby(['team.id'], as_index=False)[['team.name']].first()
df_teams['team.id'] = df_teams['team.id'].astype(str)

df_filtered = df_input.copy()

df_filtered = df_filtered.groupby(['team.id'], as_index=False).agg(
    foulwon_ratio = ('contains_foulwon', lambda x: x.sum() / x.count())
).sort_values('foulwon_ratio')

df_filtered = df_filtered.merge(df_teams, on='team.id')
df_filtered

### Correlation matrix

Fortunately there seems to be not too much correlation between the variables.

- Seconds elapsed and period naturally correlates a lot
- Forward pass, pass height and length also correlates

In [ ]:
df_filtered = df_input.copy()
num_cols = df_input.select_dtypes(include=['number']).columns.tolist()

df_filtered = df_filtered[num_cols]

df_filtered = df_filtered.corr()

plt.figure(figsize = (12,12))

ax = sns.heatmap(df_filtered,
        xticklabels=df_filtered.columns,
        yticklabels=df_filtered.columns)

### Log and standardizing

- Standardizing is applied to the numerical variables to bring them closer in terms of scale.

- As earlier was mentioned the pass length is heavily skewed so taking the log of it makes it more like a gaussian distribution which is better to use for modeling.

In [ ]:
df_filtered = df_input.copy()
df_filtered = df_filtered[df_filtered['pass.length']!=0]
df_filtered = df_filtered[['contains_foulwon', 'pass.length']]
df_filtered['pass.length_log'] = np.log(df_filtered['pass.length'] + 1)

plot_histograms_grid(df_filtered, 'contains_foulwon', bins=10)

In [ ]:
df_filtered = df_input.copy()
df_filtered = df_filtered[df_filtered['pass.length_related']!=0]
df_filtered = df_filtered[['contains_foulwon', 'pass.length_related']]
df_filtered['pass.length_related_log'] = np.log(df_filtered['pass.length_related'] + 1)

plot_histograms_grid(df_filtered, 'contains_foulwon', bins=25)

In [ ]:

df_filtered = df_input.copy()

# Stand
cols_to_standardize = [
    'seconds_elapsed',
    'manager_age_opponent',
    'manager_age_own'
]


# Initialize the scaler
scaler = StandardScaler()
df_filtered[cols_to_standardize] = scaler.fit_transform(df_filtered[cols_to_standardize])

# Log
cols_to_log = [
    'pass.length',
    'pass.length_related',
]
for col in cols_to_log:
    df_filtered[col] = np.log(df_filtered[col] + 1)

df_input = df_filtered.copy()


## Dependent variable

Our dependent variable is a boolean, which represents if during the mini possession there was a foul won by the player or not. As winning a foul is quite rare the data is heavily imbalanced.

In [ ]:
df_filtered = df_input.copy()
foulwon_ratio = df_filtered.loc[df_filtered['contains_foulwon'] == 1]['contains_foulwon'].sum() / len(df_filtered['contains_foulwon'])
print(f"Only {round(foulwon_ratio, 4)*100}% of the mini possession ends up with a foul won.")


In [ ]:
# df_filtered = df_input.copy()
df_filtered = df_filtered[['contains_foulwon']]
plot_histogram_one(df_filtered, 'contains_foulwon')

### Oversampling the minority class

In [ ]:
df_filtered = df_input.copy()

# Separate majority and minority classes
df_majority = df_filtered[df_filtered['contains_foulwon'] == 0]
df_minority = df_filtered[df_filtered['contains_foulwon'] == 1]

# Oversample minority class by duplication
df_minority_oversampled = resample(df_minority, replace=True, n_samples=round(len(df_filtered)*0.1), random_state=42)

# Combine the datasets
df_filtered = pd.concat([df_majority, df_minority_oversampled])


# df_input = df_filtered.copy()


### Undersampling the majority class

In [ ]:
categorical_cols = [
    'type.id',
    'play_pattern.id',
    'play_pattern.id_related',
    'team.id',
    'opponent_team.id',
    # 'player.id',
    'position.id',
    'position.id_related',
    'pass.body_part.id',
    'pass.body_part.id_related',
    'pass.type.id',
    'pass.type.id_related',
    'duel.outcome.id',
    'stadium.id',
    'referee.id',
    'manager_id_opponent',
    'manager_id_own',
    'manager_countryid_opponent',
    'manager_countryid_own',
    'location_x_bin',
    'location_y_bin',
    'location_y_bin_width',
    'under_pressure',
    'under_pressure_related',
    'counterpress',
    'pass.height.id',
    'pass.height.id_related'
]

numerical_cols = [
    'pass.length',
    'pass.length_related',

 ]

In [ ]:


df_filtered = df_input.copy()

df_filtered = df_filtered.drop(columns='contains_foulwon')

df_filtered = resample(df_filtered, replace=False, n_samples=round(len(df_filtered)*0.2), random_state=42)


# Initialize OneHotEncoder
OHEncoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit on multiple columns
df_filtered_encoded = OHEncoder.fit_transform(df_filtered[categorical_cols])
df_filtered_encoded = pd.DataFrame(df_filtered_encoded, columns=OHEncoder.get_feature_names_out(categorical_cols))
df_filtered = df_filtered.drop(columns=[col for col in df_filtered.columns if 'nan' in col])

df_filtered = df_filtered[numerical_cols]
df_filtered = pd.concat([df_filtered, df_filtered_encoded], axis=1)

df_filtered = df_filtered.fillna(0)

df_filtered_encoded = df_filtered.copy()


In [ ]:
df_filtered = df_filtered_encoded.copy()


# Fitting UMAP on the smaller sample
df_sample = resample(df_filtered, replace=False, n_samples=round(len(df_filtered)*0.25), random_state=42)
reducer = UMAP(random_state=42,
               n_neighbors=100,
               min_dist=0.8,
               metric='euclidean'
               )
reducer.fit(df_sample)

print('Fitting has finished')

# Transforming all the samples
df_reduced = pd.DataFrame(reducer.transform(df_filtered))

In [ ]:

clustering = HDBSCAN(gen_min_span_tree=True,
                     min_cluster_size=1000,
                     min_samples=80,
                     cluster_selection_epsilon=0.1,
                     metric='euclidean'
                     ).fit(df_reduced)
df_reduced["CLUSTER"] = abs(clustering.labels_)


# Plotting
plt.figure(figsize=(10, 8))
scatter = plt.scatter(df_reduced[0], df_reduced[1],
                      c=df_reduced['CLUSTER'],
                      cmap='viridis')
# plt.colorbar(scatter)  # Show the color bar
plt.legend()
plt.title('UMAP projection colored by CLUSTER')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.show()

In [ ]:

clustering = HDBSCAN(gen_min_span_tree=True,
                     min_cluster_size=1000,
                     min_samples=100,
                     cluster_selection_epsilon=0.1,
                     metric='euclidean'
                     ).fit(df_reduced)
df_reduced["CLUSTER"] = abs(clustering.labels_)


# Plotting
plt.figure(figsize=(10, 8))
scatter = plt.scatter(df_reduced[0], df_reduced[1],
                      c=df_reduced['CLUSTER'],
                      cmap='viridis')
# plt.colorbar(scatter)  # Show the color bar
plt.legend()
plt.title('UMAP projection colored by CLUSTER')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.show()

### Addressing class imbalance

What techniques are not used:
- SMOTE: I don't really want to use it as many of the columns contain None values. For the categorical ones most of the cases there is a dominant class and this way we would loose weight on the interesting ones if we input the most frequent class or we would make too much noise by random (or quasi random) inputing. For the numerical values assigning 0 would give them a meaning which might be misleading.

What techniques are used:
- Resampling the minority class
- Undersampling the majority class
- Using log loss (cross entropy) as evaluation metric
- Threshold tuning to be less than 0.5
- Class weighting


#### 1. Resampling

normal resampling


#### 2. Class Weighting
Instead of modifying the dataset, you can modify the learning process by assigning higher weights to the minority class.

Many machine learning models, like logistic regression, random forests, and SVM, have a class_weight parameter (balanced option in sklearn).
This helps the model focus more on the minority class without artificially altering the dataset.



#### 4. Using Proper Evaluation Metrics
Accuracy is misleading in imbalanced datasets. Instead, use:

Precision, Recall, and F1-score (better indicators of performance)
ROC-AUC or PR-AUC (Precision-Recall AUC) (better for imbalance)
Cohen’s Kappa or Matthews Correlation Coefficient (MCC) (helpful in skewed data)


#### 5. Threshold Tuning
Most models output a probability rather than a hard 0/1 decision.

By adjusting the decision threshold, you can favor recall over precision or vice versa.
For example, instead of using 0.5, lower the threshold (e.g., 0.2) to catch more positives.


#### 7. Use Specialized Models
Some algorithms handle imbalance better:

XGBoost / LightGBM / CatBoost: Have built-in parameters for imbalance (scale_pos_weight).
Bagging Classifier or Balanced Random Forests: Work well for imbalanced problems.




## Building models

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

def plot_confusion_matrices(true_vals_1, predicted_vals_1, true_vals_2, predicted_vals_2, labels=("Model 1", "Model 2")):
    """
    Plots two confusion matrices side by side.

    Parameters:
    - true_vals_1, predicted_vals_1: Ground truth and predictions for the first model.
    - true_vals_2, predicted_vals_2: Ground truth and predictions for the second model.
    - labels: Tuple of titles for each confusion matrix.
    """
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))  # 1 row, 2 columns

    # First confusion matrix
    ConfusionMatrixDisplay.from_predictions(true_vals_1, predicted_vals_1, 
                                            normalize='true', values_format=".0%", ax=axes[0])
    axes[0].set_title(labels[0])

    # Second confusion matrix
    ConfusionMatrixDisplay.from_predictions(true_vals_2, predicted_vals_2, 
                                            normalize='true', values_format=".0%", ax=axes[1])
    axes[1].set_title(labels[1])

    plt.tight_layout()
    plt.show()




### Train-test split

In [ ]:

x_main = df_filtered.drop(columns=['position'])
y_main = df_filtered['position']




X_train, X_test, y_train, y_test = train_test_split(x_main, y_main, test_size = 0.2, random_state = 0)

### Base model

In [ ]:




score_train = log_loss(y_true, y_pred_probs)
score_test = log_loss(y_test, y_pred_probs)

plot_confusion_matrices(true_vals_1, predicted_vals_1, true_vals_2, predicted_vals_2)


### Random Forest

### XGBoost

# Notes MAGYARUL

- Arra figyelj, hogy az ID-kat ne számként értelmezze
- Figyelj arra, hogy ahol lehet ne hagy None-t, pl categorical-hoz mehet valami semleges category